In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load data

In [2]:
df_initial_elos = pd.read_csv("C:/github/EECS731/major-leagues/Data/initial_elos.csv")
df_nfl_games = pd.read_csv("C:/github/EECS731/major-leagues/Data/nfl_games.csv")

In [3]:
df_initial_elos

,team,elo
0,RII,1503.947
1,STP,1300.000
2,BFF,1478.004
3,WBU,1300.000
4,RCH,1503.420
...,...,...
118,SEA,1300.000
119,CAR,1300.000
120,JAX,1300.000
121,BAL,1432.223


In [4]:
df_nfl_games

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
0,1920-09-26,1920,0,0,RII,STP,1503.947000,1300.000000,0.824651,48,0,1.0
1,1920-10-03,1920,0,0,AKR,WHE,1503.420000,1300.000000,0.824212,43,0,1.0
2,1920-10-03,1920,0,0,RCH,ABU,1503.420000,1300.000000,0.824212,10,0,1.0
3,1920-10-03,1920,0,0,DAY,COL,1493.002000,1504.908000,0.575819,14,0,1.0
4,1920-10-03,1920,0,0,RII,MUN,1516.108000,1478.004000,0.644171,45,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16269,2019-01-13,2018,0,1,NE,LAC,1640.171960,1647.624483,0.582068,41,28,1.0
16270,2019-01-13,2018,0,1,NO,PHI,1669.105861,1633.114673,0.641378,20,14,1.0
16271,2019-01-20,2018,0,1,NO,LAR,1682.450194,1648.424105,0.638772,23,26,0.0
16272,2019-01-20,2018,0,1,KC,NE,1675.286412,1661.668566,0.611248,31,37,0.0


# Transform Data

The year is important to define the team identity but the specific date is less relevant so we drop the date. Whether it is a playoff game is also dropped but the neutral site is kept because teams normally play better at home. The result is also dropped because that doesn't directly play into the score (although it indicates which team has more but I feel this will be less useful in a regression model compared to say a machine learning model).

In [5]:
df_data = df_nfl_games.drop(labels=['date','playoff','result1','elo_prob1','score1','score2'],axis=1)
df_labels = df_nfl_games.drop(labels=['date','season','neutral','playoff','team1','team2','elo1','elo2','result1','elo_prob1'],axis=1)

I tested the regression model using each elo separately and with the elo differential and the elo differential gave a better accuracy on each score by .3 points. The team name is then one-hot encoded because the teams are still important as many teams are good at offense and bad at defense or vice-versa and this will play into their score.

In [6]:
elo_differntial = df_data['elo1'] - df_data['elo2']
df_data['elo_differential'] = elo_differntial
df_data = df_data.drop(labels=['elo1','elo2'],axis=1)
df_data = pd.get_dummies(df_data,columns=['team1','team2'])

The new feature dataframe that will be fed to the regression model

In [7]:
df_features

NameError: name 'df_features' is not defined

The label for each game that will be fed to the regression model

In [ ]:
df_labels

# Build Model

In [ ]:
regressor = LinearRegression()
x = df_data.values
y = df_labels.values
regressor.fit(x,y)
guess = regressor.predict(x)

# Evaluate Model

The error is rounded because that is the value that would be used if this model were actually used as a predictor.

In [ ]:
def get_accuracy_given_error(error,limit):
    rounded_error = np.round(error)
    completely_accurate = 0
    for i in range(len(rounded_error)):
        a,b = rounded_error[i]
        if a <= limit:
            completely_accurate += 1
        if b <= limit:
            completely_accurate += 1
    return completely_accurate/(2*len(rounded_error))

In [ ]:
error = abs(y-guess)
avg_error = sum(error)/len(error)
print("Average Error:",round(sum(avg_error)/2,ndigits=3), "points")
limit_0 = round(100*get_accuracy_given_error(error,0))
limit_3 = round(100*get_accuracy_given_error(error,3))
limit_7 = round(100*get_accuracy_given_error(error,7))
print("Probability of guessing teams score correctly:", str(limit_0)+'%')
print("Probability of guessing teams score within a field goal:", str(limit_3)+'%')
print("Probability of guessing teams score within a touchdown:", str(limit_7)+'%')

8 point error is decent. This model functions okay but is probably not accurate enough to be used in a real world setting.